In [16]:
with open("data/TinyStoriesV2-GPT4-valid.txt", 'r') as f:
    corpus = f.read()[:3000]
print(corpus)

<start> u don't have to be scared of the loud dog, I'll protect you". The mole felt so safe with the little girl. She was very kind and the mole soon came to trust her. He leaned against her and she kept him safe. The mole had found his best friend. <end>
<|endoftext|>
Once upon a time, in a warm and sunny place, there was a big pit. A little boy named Tom liked to play near the pit. One day, Tom lost his red ball. He was very sad.
Tom asked his friend, Sam, to help him search for the ball. They looked high and low, but they could not find the ball. Tom said, "I think my ball fell into the pit."
Sam and Tom went close to the pit. They were scared, but they wanted to find the red ball. They looked into the pit, but it was too dark to see. Tom said, "We must go in and search for my ball."
They went into the pit to search. It was dark and scary. They could not find the ball. They tried to get out, but the pit was too deep. Tom and Sam were stuck in the pit. They called for help, but no on

In [ ]:
import regex as re

def strip_of_special_tokens(corpus, special_tokens):
    # Escape | delimiter in special tokens
    for i in range(len(special_tokens)):
        if "|" in special_tokens[i]:
            special_tokens[i] = re.escape(special_tokens[i])

    # Join special tokens into a delim for a splitting pattern
    delim = "|".join(special_tokens)
    return re.split(delim, corpus)

special_tokens = ["<|endoftext|>", "<start>", "<end>"]
print(strip_of_special_tokens(corpus, special_tokens))

['', ' u don\'t have to be scared of the loud dog, I\'ll protect you". The mole felt so safe with the little girl. She was very kind and the mole soon came to trust her. He leaned against her and she kept him safe. The mole had found his best friend. ', '\n', '\nOnce upon a time, in a warm and sunny place, there was a big pit. A little boy named Tom liked to play near the pit. One day, Tom lost his red ball. He was very sad.\nTom asked his friend, Sam, to help him search for the ball. They looked high and low, but they could not find the ball. Tom said, "I think my ball fell into the pit."\nSam and Tom went close to the pit. They were scared, but they wanted to find the red ball. They looked into the pit, but it was too dark to see. Tom said, "We must go in and search for my ball."\nThey went into the pit to search. It was dark and scary. They could not find the ball. They tried to get out, but the pit was too deep. Tom and Sam were stuck in the pit. They called for help, but no one co

In [ ]:
vocab_size = 262
num_of_merges = vocab_size - 256
vocab = {}

PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""

# Pre-tokenization
def pretokenize(corpus, ptrn):
    # Strip of special tokens
    corpus = corpus.split("")

    # Pretokenize
    counts = {}
    for t in corpus:
        for word in re.findall(ptrn, t):
            counts[word] = counts.get(word, 0) + 1
    return counts

corpus = [pretokenize(c, PAT) for c in corpus.split("<|endoftext|>")]

In [38]:
corpus.keys()

AttributeError: 'list' object has no attribute 'keys'

In [114]:
def split_to_bytes(corpus):
    counts = {}
    # Count byte pairs
    for k, v in corpus.items():
        new_key = tuple([c for c in k])
        counts[new_key] = v

    # Sort by the highest frequency
    counts = dict(sorted(counts.items(), key=lambda x: x[1], reverse=True))
    return counts

corpus = split_to_bytes(corpus)
corpus

{('n', 'e', 'w', 'e', 's', 't'): 6,
 ('l', 'o', 'w'): 5,
 ('w', 'i', 'd', 'e', 's', 't'): 3,
 ('l', 'o', 'w', 'e', 'r'): 2}

In [115]:
def count_bytepairs(corpus):
    counts = {}

    for k, v in corpus.items():
        for c1, c2 in zip(k, k[1:]):
            counts[(c1, c2)] = counts.get((c1, c2), 0) + v
    return counts

In [116]:
def get_mf_pair(counts):
    # Get the max frequency
    maxf = counts[max(counts, key=counts.get)]

    # Get the candidates with the max frequency
    candidates = [k for k, v in counts.items() if v == maxf]
    # Pick the lexicographically greater pair
    pair = max(candidates)
    return pair

In [117]:
corpus

{('n', 'e', 'w', 'e', 's', 't'): 6,
 ('l', 'o', 'w'): 5,
 ('w', 'i', 'd', 'e', 's', 't'): 3,
 ('l', 'o', 'w', 'e', 'r'): 2}

In [118]:
def merge(corpus, merge_pair):
    new_dict = {}

    for k, v in corpus.items():
        new_k = []
        b = 0
        while b < len(k):
            if b + 1 < len(k):
                c1, c2 = k[b], k[b+1]
                if c1 + c2 == merge_pair:
                    new_k.append(merge_pair)
                    b += 2
                else:
                    new_k.append(c1)
                    b += 1
            else:
                new_k.append(k[b])
                b += 1
        new_dict[tuple(new_k)] = v
    
    return new_dict

In [119]:
def pair_to_bytes(pair):
    return tuple([b.encode("utf-8") for b in pair])

In [120]:
merges = []

# Start merges
for i in range(num_of_merges):
    # Count bytepairs
    counts = count_bytepairs(corpus)
    # Get the most frequent pair
    pair = get_mf_pair(counts)
    # Add merge to merges
    pair_b = pair_to_bytes(pair)
    merges.append(pair_b)
    # Add the merge to the vocab
    merge_b = "".join(pair).encode("utf-8")
    vocab[256 + i] = merge_b
    # Apply the merge to the corpus
    corpus = merge(corpus, "".join(pair))

In [121]:
vocab

{256: b'st', 257: b'est', 258: b'ow', 259: b'low', 260: b'west', 261: b'ne'}

In [122]:
merges

[(b's', b't'),
 (b'e', b'st'),
 (b'o', b'w'),
 (b'l', b'ow'),
 (b'w', b'est'),
 (b'n', b'e')]